In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
AAPL_data_df = df = pd.read_csv("min_rating.csv")


# Review the DataFrame
AAPL_data_df

,Date,GLD,GLD % Change,VNQ,VNQ % Change,USO,USO % Change,KO,KO % Change,TSLA,...,AGG % Change,JNK,JNK % Change,50 MA,200 MA,Min Vol % Change By Weights,1 YR,5 YR,30 YR,Signal
0,2023-08-18 04:00:00+00:00,175.33,-0.000237,80.44,0.000087,72.79,0.000105,60.95,-0.000109,215.49,...,0.000072,90.840000,0.000104,0.000000,1.152515,1.159432,5.350000,4.380000,4.380000,0
1,2023-08-17 04:00:00+00:00,175.30,-0.000171,80.19,-0.003108,71.98,-0.011128,60.61,-0.005578,219.22,...,-0.002196,90.800000,-0.000440,-0.001275,1.152515,1.159432,5.360000,4.420000,4.410000,0
2,2023-08-16 04:00:00+00:00,175.57,0.001540,80.84,0.008106,71.37,-0.008475,60.48,-0.002145,225.60,...,0.001048,91.230000,0.004736,0.002630,1.152515,1.159432,5.370000,4.420000,4.380000,1
3,2023-08-15 04:00:00+00:00,176.59,0.005810,81.85,0.012494,72.62,0.017514,60.47,-0.000165,232.96,...,0.002408,91.470000,0.002631,0.004056,1.152515,1.159432,5.360000,4.360000,4.320000,1
4,2023-08-14 04:00:00+00:00,177.06,0.002662,82.78,0.011362,73.81,0.016387,60.88,0.006780,239.76,...,0.002716,91.760000,0.003170,0.004881,1.152515,1.159432,5.370000,4.360000,4.290000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937,2015-12-07 05:00:00+00:00,102.67,-0.001653,78.87,0.000888,11.72,0.003425,43.20,0.004418,231.13,...,-0.000369,102.128524,0.000000,0.001218,1.089914,1.131945,1.639022,1.929995,2.665611,1
1938,2015-12-04 05:00:00+00:00,104.02,0.013149,79.09,0.002789,12.46,0.063140,43.29,0.002083,230.38,...,-0.001476,102.128524,0.000000,0.002992,1.089556,1.131712,1.639022,1.929995,2.665611,1
1939,2015-12-03 05:00:00+00:00,101.76,-0.021727,77.61,-0.018713,12.77,0.024880,42.46,-0.019173,232.71,...,-0.003234,102.128524,0.000000,-0.009982,1.089117,1.131453,1.639022,1.929995,2.665611,0
1940,2015-12-02 05:00:00+00:00,100.69,-0.010515,78.85,0.015977,12.48,-0.022709,42.77,0.007301,231.99,...,0.007369,102.128524,0.000000,0.001392,1.088829,1.131209,1.639022,1.929995,2.665611,1


In [3]:
# Create a list of categorical variables
categorical_variables = AAPL_data_df[['Signal']].columns.tolist()

# Display the categorical variables list
categorical_variables

['Signal']

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [5]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(AAPL_data_df[categorical_variables])

In [6]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns = enc.get_feature_names_out(categorical_variables))

# Review the DataFrame
encoded_df

,Signal_0,Signal_1
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
...,...,...
1937,0.0,1.0
1938,0.0,1.0
1939,1.0,0.0
1940,0.0,1.0


In [7]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables = AAPL_data_df.select_dtypes(["int64", "float64"])
encoded_df = pd.concat([encoded_df, numerical_variables], axis = 1)

# Review the Dataframe
encoded_df

,Signal_0,Signal_1,GLD,GLD % Change,VNQ,VNQ % Change,USO,USO % Change,KO,KO % Change,...,AGG % Change,JNK,JNK % Change,50 MA,200 MA,Min Vol % Change By Weights,1 YR,5 YR,30 YR,Signal
0,1.0,0.0,175.33,-0.000237,80.44,0.000087,72.79,0.000105,60.95,-0.000109,...,0.000072,90.840000,0.000104,0.000000,1.152515,1.159432,5.350000,4.380000,4.380000,0
1,1.0,0.0,175.30,-0.000171,80.19,-0.003108,71.98,-0.011128,60.61,-0.005578,...,-0.002196,90.800000,-0.000440,-0.001275,1.152515,1.159432,5.360000,4.420000,4.410000,0
2,0.0,1.0,175.57,0.001540,80.84,0.008106,71.37,-0.008475,60.48,-0.002145,...,0.001048,91.230000,0.004736,0.002630,1.152515,1.159432,5.370000,4.420000,4.380000,1
3,0.0,1.0,176.59,0.005810,81.85,0.012494,72.62,0.017514,60.47,-0.000165,...,0.002408,91.470000,0.002631,0.004056,1.152515,1.159432,5.360000,4.360000,4.320000,1
4,0.0,1.0,177.06,0.002662,82.78,0.011362,73.81,0.016387,60.88,0.006780,...,0.002716,91.760000,0.003170,0.004881,1.152515,1.159432,5.370000,4.360000,4.290000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937,0.0,1.0,102.67,-0.001653,78.87,0.000888,11.72,0.003425,43.20,0.004418,...,-0.000369,102.128524,0.000000,0.001218,1.089914,1.131945,1.639022,1.929995,2.665611,1
1938,0.0,1.0,104.02,0.013149,79.09,0.002789,12.46,0.063140,43.29,0.002083,...,-0.001476,102.128524,0.000000,0.002992,1.089556,1.131712,1.639022,1.929995,2.665611,1
1939,1.0,0.0,101.76,-0.021727,77.61,-0.018713,12.77,0.024880,42.46,-0.019173,...,-0.003234,102.128524,0.000000,-0.009982,1.089117,1.131453,1.639022,1.929995,2.665611,0
1940,0.0,1.0,100.69,-0.010515,78.85,0.015977,12.48,-0.022709,42.77,0.007301,...,0.007369,102.128524,0.000000,0.001392,1.088829,1.131209,1.639022,1.929995,2.665611,1


In [8]:
# Define the target set y using the "Target - B/H/S (based on close - daily % change") column
y = encoded_df["Signal"]

# Display a sample of y
y

0       0
1       0
2       1
3       1
4       1
       ..
1937    1
1938    1
1939    0
1940    1
1941    1
Name: Signal, Length: 1942, dtype: int64

In [9]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns = "Signal")

# Review the features DataFrame
X.dtypes

Signal_0                       float64
Signal_1                       float64
GLD                            float64
GLD % Change                   float64
VNQ                            float64
VNQ % Change                   float64
USO                            float64
USO % Change                   float64
KO                             float64
KO % Change                    float64
TSLA                           float64
TSLA % Change                  float64
AAPL                           float64
AAPL % Change                  float64
AGG                            float64
AGG % Change                   float64
JNK                            float64
JNK % Change                   float64
50 MA                          float64
200 MA                         float64
Min Vol % Change By Weights    float64
1 YR                           float64
5 YR                           float64
30 YR                          float64
dtype: object

In [10]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
model = XGBClassifier(
    n_estimators=100,  # Number of boosting rounds (trees)
    max_depth=3,       # Maximum depth of each tree
    learning_rate=0.1, # Step size at each iteration
    random_state=42
)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
model = XGBClassifier(
    n_estimators=100,  # Number of boosting rounds (trees)
    max_depth=3,       # Maximum depth of each tree
    learning_rate=0.1, # Step size at each iteration
    random_state=42
)

In [15]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [16]:
y_pred = model.predict(X_test)

In [17]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00
